In [0]:
dbutils.fs.ls("/FileStore/tables/")

Out[1]: [FileInfo(path='dbfs:/FileStore/tables/FaultDataset.csv', name='FaultDataset.csv', size=1703184, modificationTime=1682262899000),
 FileInfo(path='dbfs:/FileStore/tables/Occupancy_Detection_Data.csv', name='Occupancy_Detection_Data.csv', size=50968, modificationTime=1682258711000),
 FileInfo(path='dbfs:/FileStore/tables/Solutions___Recommender_System.zip', name='Solutions___Recommender_System.zip', size=82913, modificationTime=1682326871000),
 FileInfo(path='dbfs:/FileStore/tables/account-models/', name='account-models/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/account-models1/', name='account-models1/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/accounts/', name='accounts/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/activations/', name='activations/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/activations-1.zip', name='activations-1.zip', size=8411369, modificationTime=168224

In [0]:
fileroot = "clinicaltrial_2021"
dbutils.fs.cp("/FileStore/tables/" + fileroot + ".zip", "file:/tmp/")

Out[3]: True

In [0]:
import os
os.environ['fileroot'] = fileroot

In [0]:
%sh
unzip -d /tmp /tmp/$fileroot.zip

Archive:  /tmp/clinicaltrial_2021.zip
  inflating: /tmp/clinicaltrial_2021.csv  


In [0]:
dbutils.fs.mkdirs("/FileStore/tables/" + fileroot)

Out[6]: True

In [0]:
dbutils.fs.mv("file:/tmp/" + fileroot + ".csv", "/FileStore/tables/" + fileroot + ".csv", True)

Out[7]: True

In [0]:
dbutils.fs.ls("file:/tmp/")

Out[8]: [FileInfo(path='file:/tmp/hsperfdata_root/', name='hsperfdata_root/', size=4096, modificationTime=1682327406363),
 FileInfo(path='file:/tmp/Rserv/', name='Rserv/', size=4096, modificationTime=1682327432150),
 FileInfo(path='file:/tmp/systemd-private-e8863be177ec40db8f440b44d28df45f-ntp.service-78iXDi/', name='systemd-private-e8863be177ec40db8f440b44d28df45f-ntp.service-78iXDi/', size=4096, modificationTime=1682327308826),
 FileInfo(path='file:/tmp/.font-unix/', name='.font-unix/', size=4096, modificationTime=1682327308782),
 FileInfo(path='file:/tmp/.ICE-unix/', name='.ICE-unix/', size=4096, modificationTime=1682327308782),
 FileInfo(path='file:/tmp/driver-daemon-params', name='driver-daemon-params', size=19, modificationTime=1682327340617),
 FileInfo(path='file:/tmp/custom-spark.conf', name='custom-spark.conf', size=548, modificationTime=1682327323653),
 FileInfo(path='file:/tmp/systemd-private-e8863be177ec40db8f440b44d28df45f-apache2.service-vyqZgh/', name='systemd-private-e8

In [0]:
dbutils.fs.ls("/FileStore/tables" )

Out[9]: [FileInfo(path='dbfs:/FileStore/tables/FaultDataset.csv', name='FaultDataset.csv', size=1703184, modificationTime=1682262899000),
 FileInfo(path='dbfs:/FileStore/tables/Occupancy_Detection_Data.csv', name='Occupancy_Detection_Data.csv', size=50968, modificationTime=1682258711000),
 FileInfo(path='dbfs:/FileStore/tables/Solutions___Recommender_System.zip', name='Solutions___Recommender_System.zip', size=82913, modificationTime=1682326871000),
 FileInfo(path='dbfs:/FileStore/tables/account-models/', name='account-models/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/account-models1/', name='account-models1/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/accounts/', name='accounts/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/activations/', name='activations/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/activations-1.zip', name='activations-1.zip', size=8411369, modificationTime=168224

In [0]:
clinicaltrial_2021RDD = sc.textFile("/FileStore/tables/clinicaltrial_2021.csv")
pharmaRDD = sc.textFile("/FileStore/tables/pharma.csv")

In [0]:
#Question 1
removeHeader = clinicaltrial_2021RDD.first()
clinicaltrial_2021RDD = clinicaltrial_2021RDD.filter(lambda line: line != removeHeader)#RDD
clinicaltrial_2021RDD = sc.textFile("/FileStore/tables/clinicaltrial_2021.csv")
distinctRDD = clinicaltrial_2021RDD.map(lambda line: line.split("|")[0]).distinct().count()
print("distinct:", distinctRDD)

distinct: 387262


In [0]:
#Question 2
clinicaltrial_2021 = sc.textFile("/FileStore/tables/clinicaltrial_2021.csv")
clinicaltrial_2021_split = clinicaltrial_2021.map(lambda line: line.split("|"))
clinicaltrial_2021_type = clinicaltrial_2021_split.map(lambda x: x[5])
clinicaltrial_2021_join = clinicaltrial_2021_type.map(lambda word: (word,1))
clinicaltrial_2021_count = clinicaltrial_2021_join.reduceByKey(lambda x,y: x+y)
clinicaltrial_2021_show = clinicaltrial_2021_count.sortBy(lambda x: [-x[1],x[0]])
clinicaltrial_2021_show.take(4)

[('Interventional', 301472),
 ('Observational', 77540),
 ('Observational [Patient Registry]', 8180),
 ('Expanded Access', 69)]

In [0]:
#Question 3
clinicaltrial_2021 = sc.textFile("/FileStore/tables/clinicaltrial_2021.csv")
clinicaltrial_2021_split = clinicaltrial_2021.map(lambda line: line.split("|"))
clinicaltrial_2021_term = clinicaltrial_2021_split.map(lambda x: x[7])
trial1 = clinicaltrial_2021_term.first()
clinicaltrial_2021_filter = clinicaltrial_2021_term.filter(lambda x: x!= trial1)
clinicaltrial_2021_data = clinicaltrial_2021_filter.flatMap(lambda z: z.split(','))
clinicaltrial_2021_join = clinicaltrial_2021_data.map(lambda word: (word,1))
clinicaltrial_2021_count = clinicaltrial_2021_join.reduceByKey(lambda x,y: x+y)
clinicaltrial_2021_sort = clinicaltrial_2021_count.sortBy(lambda x: [-x[1],x[0]])
trial2 = clinicaltrial_2021_sort.first()
clinicaltrial_2021_RDD = clinicaltrial_2021_sort.filter(lambda x: x!=trial2)
clinicaltrial_2021_RDD.take(5)

[('Carcinoma', 13389),
 ('Diabetes Mellitus', 11080),
 ('Neoplasms', 9371),
 ('Breast Neoplasms', 8640),
 ('Syndrome', 8032)]

In [0]:
#Question 4
pharma_data = sc.textFile("/FileStore/tables/pharma.csv")
parent_company = pharma_data.map(lambda x: x.split(",")[1]).map(lambda line: line.replace('"',''))
sponsor_data = clinicaltrial_2021.map(lambda x: x.split("|")[1])
RDD_1 = sponsor_data.subtract(parent_company)
join_data = RDD_1.map(lambda x: (x,1))
RDD_add = join_data.reduceByKey(lambda x,y: x+y)
RDD_result = RDD_add.sortBy(lambda x: x[1], ascending = False)
RDD_result.take(10)

[('National Cancer Institute (NCI)', 3218),
 ('M.D. Anderson Cancer Center', 2414),
 ('Assistance Publique - Hôpitaux de Paris', 2369),
 ('Mayo Clinic', 2300),
 ('Merck Sharp & Dohme Corp.', 2243),
 ('Assiut University', 2154),
 ('Novartis Pharmaceuticals', 2088),
 ('Massachusetts General Hospital', 1971),
 ('Cairo University', 1928),
 ('Hoffmann-La Roche', 1828)]

In [0]:
#Question 5
clinicaltrial_2021_split = clinicaltrial_2021.map(lambda line: line.split("|"))
clinicaltrial_2021_data = clinicaltrial_2021_split.map(lambda x: (x[2], x[4]))
clinicaltrial_2021_filter = clinicaltrial_2021_data.filter(lambda x: 'Completed' in x)
clinicaltrial_2021_map = clinicaltrial_2021_filter.map(lambda x: x[1])
clinicaltrial_2021_filter1 = clinicaltrial_2021_map.filter(lambda x: '2021' in x)
clinicaltrial_2021_map1 = clinicaltrial_2021_filter1.map(lambda x: (x,1))
clinicaltrial_2021_red = clinicaltrial_2021_map1.reduceByKey(lambda x,y:x+y)

from datetime import datetime
def months(date):
    date = datetime.strptime(date,'%b %Y')
    monthly_2021 = date.month
    return monthly_2021
clinicaltrial_2021_map2 = clinicaltrial_2021_red.map(lambda line: (line[0], months(line[0]),line[1]))
clinicaltrial_2021_sort = clinicaltrial_2021_map2.sortBy(lambda line: line[1])
clinicaltrial_2021_map3 = clinicaltrial_2021_sort.map(lambda line: (line[0].split(' ')[0],line[2]))
clinicaltrial_2021_map3.take(12)

[('Jan', 1131),
 ('Feb', 934),
 ('Mar', 1227),
 ('Apr', 967),
 ('May', 984),
 ('Jun', 1094),
 ('Jul', 819),
 ('Aug', 700),
 ('Sep', 528),
 ('Oct', 187)]